# 6º Período ✅

# Pré-processamento dos dados


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# nltk.download('stopwords')
# nltk.download('punkt')
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam, Nadam
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import string 
import re
from keras.preprocessing.text import Tokenizer
from sklearn.svm import SVC # SVM model algorithm
from sklearn.metrics import accuracy_score # evalution metric
from sklearn.metrics import confusion_matrix # evalution metric
from sklearn.model_selection import train_test_split # splitting the data
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import mean_squared_error, r2_score, classification_report
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_aux = pd.read_csv('/content/drive/MyDrive/IEEE/Musical_instruments_reviews.csv')
df_aux.head(3)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"
2,A195EZSQDW3E21,1384719342,"Rick Bennette ""Rick Bennette""","[1, 1]",The primary job of this device is to block the...,5.0,It Does The Job Well,1377648000,"08 28, 2013"


In [ ]:
# using only two columns of the dataset: "reviewText" and "overall"
df = df_aux[['reviewText', 'overall']]
df.head(3)

,reviewText,overall
0,"Not much to write about here, but it does exac...",5.0
1,The product does exactly as it should and is q...,5.0
2,The primary job of this device is to block the...,5.0


In [ ]:
# drop na values -- there is no Na values
df.isna().sum()

reviewText    7
overall       0
dtype: int64

In [ ]:
# make the witdh of the columns bigger
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [ ]:
df.head(2)

,reviewText,overall
0,"Not much to write about here, but it does exactly what it's supposed to. filters out the pop sounds. now my recordings are much more crisp. it is one of the lowest prices pop filters on amazon so might as well buy it, they honestly work the same despite their pricing,",5.0
1,"The product does exactly as it should and is quite affordable.I did not realized it was double screened until it arrived, so it was even better than I had expected.As an added bonus, one of the screens carries a small hint of the smell of an old grape candy I used to buy, so for reminiscent's sake, I cannot stop putting the pop filter next to my nose and smelling it after recording. :DIf you needed a pop filter, this will work just as well as the expensive ones, and it may even come with a pleasing aroma like mine did!Buy this product! :]",5.0


In [ ]:
df.shape

(10261, 2)

## Remove punctuation

In [ ]:
# remove the punctuation
string.punctuation

# function that removes the punctuation 
def remove_punc(text):
  no_punc = ''.join([i for i in text if i not in string.punctuation])
  return no_punc

  # puctuation free text had to be saved

In [ ]:
# remove the punctuation from the columns that store texts

df['reviewText'] = df['reviewText'].apply(lambda x: remove_punc(str(x)))
df['overall'] = df['overall'].apply(lambda x: remove_punc(str(x)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df.head(3)

,reviewText,overall
0,Not much to write about here but it does exactly what its supposed to filters out the pop sounds now my recordings are much more crisp it is one of the lowest prices pop filters on amazon so might as well buy it they honestly work the same despite their pricing,50
1,The product does exactly as it should and is quite affordableI did not realized it was double screened until it arrived so it was even better than I had expectedAs an added bonus one of the screens carries a small hint of the smell of an old grape candy I used to buy so for reminiscents sake I cannot stop putting the pop filter next to my nose and smelling it after recording DIf you needed a pop filter this will work just as well as the expensive ones and it may even come with a pleasing aroma like mine didBuy this product,50
2,The primary job of this device is to block the breath that would otherwise produce a popping sound while allowing your voice to pass through with no noticeable reduction of volume or high frequencies The double cloth filter blocks the pops and lets the voice through with no coloration The metal clamp mount attaches to the mike stand secure enough to keep it attached The goose neck needs a little coaxing to stay where you put it,50


## Lowering case

In [ ]:
# lowering the text
columns = ['reviewText',	'overall']

def lower_case():
  for i in columns:
    df[i] = df[i].apply(lambda x: str(x).lower())

lower_case()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df.head()

,reviewText,overall
0,not much to write about here but it does exactly what its supposed to filters out the pop sounds now my recordings are much more crisp it is one of the lowest prices pop filters on amazon so might as well buy it they honestly work the same despite their pricing,50
1,the product does exactly as it should and is quite affordablei did not realized it was double screened until it arrived so it was even better than i had expectedas an added bonus one of the screens carries a small hint of the smell of an old grape candy i used to buy so for reminiscents sake i cannot stop putting the pop filter next to my nose and smelling it after recording dif you needed a pop filter this will work just as well as the expensive ones and it may even come with a pleasing aroma like mine didbuy this product,50
2,the primary job of this device is to block the breath that would otherwise produce a popping sound while allowing your voice to pass through with no noticeable reduction of volume or high frequencies the double cloth filter blocks the pops and lets the voice through with no coloration the metal clamp mount attaches to the mike stand secure enough to keep it attached the goose neck needs a little coaxing to stay where you put it,50
3,nice windscreen protects my mxl mic and prevents pops only thing is that the gooseneck is only marginally able to hold the screen in position and requires careful positioning of the clamp to avoid sagging,50
4,this pop filter is great it looks and performs like a studio filter if youre recording vocals this will eliminate the pops that gets recorded when you sing,50


## Tokenization with tensorflow

In [ ]:
tokenizer = Tokenizer()

In [ ]:
# we need to tokenizer the 'reviewText'
token_feat = df['reviewText'].to_list()

In [ ]:
tokenizer.fit_on_texts(token_feat)
tokenizer_next = tokenizer.texts_to_sequences(token_feat)

df_token = pd.DataFrame(tokenizer_next)
df_token.head(3)

,0,1,2,3,4,5,6,7,8,9,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,20,70.0,5.0,1725.0,64.0,289.0,17.0,6.0,83.0,370.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,118.0,83.0,370.0,21.0,6.0,172.0,3.0,7.0,239.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1756.0,215.0,8.0,10.0,431.0,7.0,5.0,1609.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# print(len(sequences))

* Each element in the array is a string tokenized. 

## Stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# stop words removal
stopwords = stopwords.words('english')
stopwords[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his']

In [ ]:
# remove stopwords from tokenized text
df['reviewText']= df['reviewText'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords]))
df.head(3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewText,overall
0,much write exactly supposed filters pop sounds recordings much crisp one lowest prices pop filters amazon might well buy honestly work despite pricing,50
1,product exactly quite affordablei realized double screened arrived even better expectedas added bonus one screens carries small hint smell old grape candy used buy reminiscents sake cannot stop putting pop filter next nose smelling recording dif needed pop filter work well expensive ones may even come pleasing aroma like mine didbuy product,50
2,primary job device block breath would otherwise produce popping sound allowing voice pass noticeable reduction volume high frequencies double cloth filter blocks pops lets voice coloration metal clamp mount attaches mike stand secure enough keep attached goose neck needs little coaxing stay put,50


In [ ]:
df.head()

,reviewText,overall
0,much write exactly supposed filters pop sounds recordings much crisp one lowest prices pop filters amazon might well buy honestly work despite pricing,50
1,product exactly quite affordablei realized double screened arrived even better expectedas added bonus one screens carries small hint smell old grape candy used buy reminiscents sake cannot stop putting pop filter next nose smelling recording dif needed pop filter work well expensive ones may even come pleasing aroma like mine didbuy product,50
2,primary job device block breath would otherwise produce popping sound allowing voice pass noticeable reduction volume high frequencies double cloth filter blocks pops lets voice coloration metal clamp mount attaches mike stand secure enough keep attached goose neck needs little coaxing stay put,50
3,nice windscreen protects mxl mic prevents pops thing gooseneck marginally able hold screen position requires careful positioning clamp avoid sagging,50
4,pop filter great looks performs like studio filter youre recording vocals eliminate pops gets recorded sing,50


## Lemmatization

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
lemmatizer = WordNetLemmatizer()
df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewText,overall
0,much write exactly supposed filter pop sound recording much crisp one lowest price pop filter amazon might well buy honestly work despite pricing,50
1,product exactly quite affordablei realized double screened arrived even better expectedas added bonus one screen carry small hint smell old grape candy used buy reminiscents sake cannot stop putting pop filter next nose smelling recording dif needed pop filter work well expensive one may even come pleasing aroma like mine didbuy product,50
2,primary job device block breath would otherwise produce popping sound allowing voice pas noticeable reduction volume high frequency double cloth filter block pop let voice coloration metal clamp mount attache mike stand secure enough keep attached goose neck need little coaxing stay put,50
3,nice windscreen protects mxl mic prevents pop thing gooseneck marginally able hold screen position requires careful positioning clamp avoid sagging,50
4,pop filter great look performs like studio filter youre recording vocal eliminate pop get recorded sing,50


* Frequently, the stemming generate some root words with no sense. And this useful had to be evaluated, in this case, is just a example of implementation of 'stemming process' in a text.

* For example: 1. crazy - crazi, 2. available - avail.

* The lemmatization process is a better choice than stemming, because the first stems the word but makes sure that it doesn't lose its meaning. 

## Stemming 

In [ ]:
porter = PorterStemmer()

In [ ]:
# get stemming words
porter_stemmer = PorterStemmer()

df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([porter_stemmer.stem(word) for word in x.split()]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
df.head(3)

,reviewText,overall
0,much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price,50
1,product exactli quit affordablei realiz doubl screen arriv even better expecteda ad bonu one screen carri small hint smell old grape candi use buy reminisc sake cannot stop put pop filter next nose smell record dif need pop filter work well expens one may even come pleas aroma like mine didbuy product,50
2,primari job devic block breath would otherwis produc pop sound allow voic pa notic reduct volum high frequenc doubl cloth filter block pop let voic color metal clamp mount attach mike stand secur enough keep attach goos neck need littl coax stay put,50


# Training a classifier using Support Vector Machine (SVM)

In [ ]:
clf = TfidfVectorizer(max_features = 4000, ngram_range = (1, 3))
x = clf.fit_transform(df['reviewText'])

In [ ]:
x
y = df['overall']

In [ ]:
model = SVC(kernel = 'rbf')

In [ ]:
smote = SMOTE()

In [ ]:
x_new, y_new = smote.fit_resample(x, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_new, y_new, test_size = 0.2, random_state = 0)

In [ ]:
X_train, y_train

(<27752x4000 sparse matrix of type '<class 'numpy.float64'>'
 	with 2071186 stored elements in Compressed Sparse Row format>, 25647    30
 28916    30
 34596    40
 31752    40
 5210     50
          ..
 20757    20
 32103    40
 30403    40
 21243    20
 2732     50
 Name: overall, Length: 27752, dtype: object)

In [ ]:
clf2 = SVC()
clf2 = clf2.fit(X_train, y_train)

In [ ]:
y_pred = clf2.predict(X_test)

In [ ]:
score = r2_score(y_test, y_pred)
score

0.9697254580500859

# Training a RNN

In [ ]:
df.head(3)

,reviewText,overall
0,much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price,50
1,product exactli quit affordablei realiz doubl screen arriv even better expecteda ad bonu one screen carri small hint smell old grape candi use buy reminisc sake cannot stop put pop filter next nose smell record dif need pop filter work well expens one may even come pleas aroma like mine didbuy product,50
2,primari job devic block breath would otherwis produc pop sound allow voic pa notic reduct volum high frequenc doubl cloth filter block pop let voic color metal clamp mount attach mike stand secur enough keep attach goos neck need littl coax stay put,50


In [ ]:
texts = df['reviewText'].values

In [ ]:
texts[0:2]

array(['much write exactli suppos filter pop sound record much crisp one lowest price pop filter amazon might well buy honestli work despit price',
       'product exactli quit affordablei realiz doubl screen arriv even better expecteda ad bonu one screen carri small hint smell old grape candi use buy reminisc sake cannot stop put pop filter next nose smell record dif need pop filter work well expens one may even come pleas aroma like mine didbuy product'],
      dtype=object)

In [ ]:
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

sequences[0:1]

[[23,
  970,
  314,
  444,
  512,
  376,
  3,
  58,
  23,
  4,
  17,
  376,
  512,
  192,
  173,
  12,
  43,
  928,
  10,
  17]]

In [ ]:
pad = pad_sequences(sequences, maxlen = 90, dtype = 'int32', padding = 'post', truncating = 'post')

pad[10222]

array([771,  17, 238,   5, 300,   5,  24,  68,  42, 464, 226, 262, 119,
       737,  60,  40,   4,   2,  95,   5,  55, 144, 617,   5,  68,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [ ]:
# one hot encoding to classifier correctly
X = pad
y = LabelBinarizer().fit_transform(df['overall'])

print(X, y)

[[ 23 970 314 ...   0   0   0]
 [ 47 314 153 ...   0   0   0]
 [140 298  15 ...   0   0   0]
 ...
 [ 39 595   5 ...   0   0   0]
 [ 12  62 471 ... 799   5 220]
 [  5  18 153 ...   0   0   0]] [[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 ...
 [0 0 0 1 0]
 [0 0 0 1 0]
 [0 0 0 1 0]]


* The one hot encoding is necessary to equalize the output of the 5 neurons to the final output of the RNN.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# the model with the Embedding layer
embedding_vector_len = 32

el_model = Sequential()

el_model.add(layers.Embedding(input_dim=1000, output_dim=64))
el_model.add(layers.SimpleRNN(128))
el_model.add(Dense(64, activation = 'relu'))
el_model.add(Dense(5, activation = "softmax"))

In [ ]:
el_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
el_model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = 30, batch_size = 70)

Epoch 1/30
118/118 [==============================] - 10s 66ms/step - loss: 0.3493 - accuracy: 0.6601 - val_loss: 0.3297 - val_accuracy: 0.6693
Epoch 2/30
118/118 [==============================] - 8s 64ms/step - loss: 0.3266 - accuracy: 0.6779 - val_loss: 0.3258 - val_accuracy: 0.6693
Epoch 3/30
118/118 [==============================] - 8s 71ms/step - loss: 0.3253 - accuracy: 0.6779 - val_loss: 0.3298 - val_accuracy: 0.6693
Epoch 4/30
118/118 [==============================] - 7s 63ms/step - loss: 0.3245 - accuracy: 0.6773 - val_loss: 0.3269 - val_accuracy: 0.6693
Epoch 5/30
118/118 [==============================] - 7s 63ms/step - loss: 0.3231 - accuracy: 0.6817 - val_loss: 0.3318 - val_accuracy: 0.6693
Epoch 6/30
118/118 [==============================] - 7s 62ms/step - loss: 0.3175 - accuracy: 0.6910 - val_loss: 0.3331 - val_accuracy: 0.6663
Epoch 7/30
118/118 [==============================] - 7s 62ms/step - loss: 0.3118 - accuracy: 0.6962 - val_loss: 0.3303 - val_accuracy: 0.665

* The RNN had an val_accuracy around 65%. It's relatively acceptable in this case, other methods could improve this performance.